In [1]:
# %%
# Install required packages (run only if not already installed)
!pip install -q sentence-transformers pinecone-client python-dotenv pandas ollama

In [2]:
# %%
# Import libraries
import os
from dotenv import load_dotenv
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import ollama


In [3]:

# %%
# Load API keys and environment variables from .env file
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")
assert PINECONE_API_KEY and PINECONE_ENV, "Pinecone API key or environment not found in .env!"


In [4]:
# %%
# Initialize Pinecone client and connect to your index
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "funding-search"
index = pc.Index(index_name)


In [5]:

# %%
# Load the embedding model ONCE for efficiency
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


In [6]:
# %%
# Function to retrieve top-k relevant funding programs from Pinecone
def retrieve_docs(query, top_k=5, model=embedding_model):
    """
    Retrieve top-k relevant funding programs from Pinecone for a given query.

    Args:
        query (str): The user's search query.
        top_k (int): Number of top results to retrieve.
        model (SentenceTransformer): The embedding model to use.

    Returns:
        list of str: Formatted funding program descriptions for LLM context.
    """
    # Encode the query to get its vector
    query_vector = model.encode(query).tolist()
    # Query Pinecone for top-k matches
    search_results = index.query(vector=query_vector, top_k=top_k, include_metadata=True)
    docs = []
    for match in search_results["matches"]:
        meta = match["metadata"]
        doc_text = (
            f"Name: {meta.get('name', '')}\n"
            f"Description: {meta.get('description', '')}\n"
            f"Eligibility: {meta.get('eligibility', '')}\n"
            f"Amount: {meta.get('amount', '')}\n"
            f"Domain: {meta.get('domain', '')}\n"
            f"Location: {meta.get('location', '')}\n"
            f"Procedure: {meta.get('procedure', '')}\n"
            f"URL: {meta.get('url', '')}\n"
            f"Source: {meta.get('source', '')}"
        )
        docs.append(doc_text)
    return docs


In [ ]:
# # %%
# # Function to generate a RAG answer using Ollama (Llama 3 or Mixtral)
# def rag_answer(query, top_k=5, model_name='llama3'):
#     """
#     Generate a RAG answer for a user's query using Pinecone retrieval and Ollama LLM.

#     Args:
#         query (str): The user's question.
#         top_k (int): Number of top documents to retrieve for context.
#         model_name (str): The Ollama model to use ('llama3' or 'mixtral').

#     Returns:
#         str: The generated answer from the LLM.
#     """
#     docs = retrieve_docs(query, top_k=top_k)
#     context = "\n\n".join(docs)
#     prompt = (
#         f"You are a funding expert. Given the following funding programs:\n\n"
#         f"{context}\n\n"
#         f"Answer the user's question, using only the provided information. "
#         f"If the answer is not in the data, say so.\n"
#         f"User question: {query}\n"
#         f"Answer:"
#     )
#     # Call Ollama's local LLM
#     response = ollama.generate(model=model_name, prompt=prompt)
#     return response['response']



In [12]:
from ollama import chat

def rag_answer(query, top_k=5, model_name="llama3.2"):
    """
    Generate a RAG answer using Pinecone retrieval and Ollama LLM (chat API).
    """
    docs = retrieve_docs(query, top_k=top_k)
    context = "\n\n".join(docs)
    prompt = (
        f"You are a funding expert. Given the following funding programs:\n\n"
        f"{context}\n\n"
        f"Answer the user's question, using only the provided information. "
        f"If the answer is not in the data, say so.\n"
        f"User question: {query}\n"
        f"Answer:"
    )
    response = chat(
        model=model_name,
        messages=[{'role': 'user', 'content': prompt}]
    )
    # For latest ollama-python, the answer is in response['message']['content']
    return response['message']['content']


In [ ]:
# %%
# Example usage: Ask a question and get a RAG answer
query = "What AI funding is available for SMEs in Germany above 50,000 euros?"
answer = rag_answer(query, top_k=5, model_name="llama3.2")  # or model_name="mixtral"
print("Q:", query)
print("A:", answer)



Q: What AI funding is available for SMEs in Germany above 50,000 euros?
A: Based on the provided information, there are two funding programs offered by the Investitions- und Strukturbank Rheinland-Pfalz (ISB):

1. Research and development (InnoTop): This program provides non-repayable grants for small and medium-sized enterprises (SMEs) in Rhineland-Palatinate. The maximum grant amounts are EUR 52,500 for feasibility studies and EUR 500,000 for R&D projects.
2. Venture capital/investments: This program provides equity capital to SMEs in the form of direct investments or typical silent partnerships. The maximum investment amount is EUR 500,000.

However, there is no specific funding available above EUR 50,000 for SMEs in Germany. The information provided only mentions these two programs and their corresponding grant amounts, but does not mention any other funding opportunities that might be available.

Therefore, the answer to your question is: None of the mentioned AI funding programs 

In [14]:
# %%
# Optional: Table-based semantic search for UI or analysis
def semantic_search(query, top_k=5, model=embedding_model):
    """
    Return a DataFrame of the top-k relevant funding programs for a given query.
    """
    query_vector = model.encode(query).tolist()
    search_results = index.query(vector=query_vector, top_k=top_k, include_metadata=True)
    results = [
        {
            "name": match["metadata"].get("name", ""),
            "description": match["metadata"].get("description", ""),
            "eligibility": match["metadata"].get("eligibility", ""),
            "amount": match["metadata"].get("amount", ""),
            "domain": match["metadata"].get("domain", ""),
            "location": match["metadata"].get("location", ""),
            "procedure": match["metadata"].get("procedure", ""),
            "url": match["metadata"].get("url", ""),
            "source": match["metadata"].get("source", ""),
            "score": match["score"]
        }
        for match in search_results["matches"]
    ]
    return pd.DataFrame(results)


In [15]:
# %%
# Example: Display search results as a table
results_df = semantic_search("AI funding support for tech startups in Germany", top_k=5)
display(results_df)


,name,description,eligibility,amount,domain,location,procedure,url,source,score
0,Promotion of innovation assistants in small an...,Administrative regulation of the Ministry of E...,"Technology-oriented, small and medium-sized en...",Funding is provided as project funding in the ...,domain information not found,location information not found,Applications are submitted directly to the Inv...,https://isb.rlp.de/foerderung/245i.html,isb,0.567497
1,Promotion of innovation assistants in small an...,Administrative regulation of the Ministry of E...,,,,,,https://isb.rlp.de/foerderung/245i.html,isb,0.567497
2,Promotion of innovation assistants in small an...,Administrative regulation of the Ministry of E...,"Technology-oriented, small and medium-sized en...",Funding is provided as project funding in the ...,domain information not found,location information not found,Applications are submitted directly to the Inv...,https://isb.rlp.de/foerderung/245i.html,isb,0.567497
3,Projects within the framework of the European ...,If you would like to participate as a German s...,"University, research institution, company",amount information not found,"Research & innovation (topic-specific), digita...",Nationwide,procedure information not found,https://www.foerderdatenbank.de/FDB/Content/DE...,foerderdatenbank,0.540310
4,Projects within the framework of the European ...,If you would like to participate as a German s...,"University, research institution, company",amount information not found,"Research & innovation (topic-specific), digita...",Nationwide,procedure information not found,https://www.foerderdatenbank.de/FDB/Content/DE...,foerderdatenbank,0.540310
